# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from scipy.stats import unitary_group
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([45])

CLASS_TO_IGNORE = 2  # third class, 0, 1, 2
INITIALIZATION_METHOD = 'Angle'
EPOCHS = 50
TEST_SIZE = 0.2
STEP_SIZE = 0.3
STEPS = int((1-TEST_SIZE)*100*EPOCHS)

# Import dataset

In [3]:
X, y = load_iris(return_X_y=True)

if CLASS_TO_IGNORE == 0:
    # Drop first class
    X = X[50:]
    y = y[50:]
    y_scaled = np.where(y == 1, -1, y)
    y_scaled = np.where(y_scaled == 2, 1, y_scaled)

elif CLASS_TO_IGNORE == 1:
    # Drop second class
    X = np.delete(X, range(50, 100))
    y = np.delete(y, range(50, 100))
    y_scaled = np.where(y == 0, -1, y)
    y_scaled = np.where(y_scaled == 2, 1, y_scaled)

else:
    # Drop third class
    X = X[:100]
    y = y[:100]
    y_scaled = np.where(y == 0, -1, y)

In [4]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=TEST_SIZE, shuffle=True)

# Circuit creation

In [5]:
device = qml.device("default.qubit", wires=4)

In [6]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(4), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(4), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])
    ArbitraryUnitary(params[15:30], wires=[2, 3])

    # Second layer
    ArbitraryUnitary(params[30:], wires=[1, 2])

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [7]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998]

Example features: [ 1.29839254 -1.67737625  1.06705859  0.91406997]

Expectation value: 0.9999999999999967

 0: ──H──RZ(0.375)──H──RX(1.57)──RZ(0.951)──RX(-1.57)──RZ(0.732)──╭RZ(0.599)──H──────────────────────╭RZ(0.156)──H──RX(1.57)──╭RZ(0.0581)──RX(-1.57)──RX(1.57)──╭RZ(0.866)──RX(-1.57)────────────╭RZ(0.601)──H────────────────────────────────────────────────────╭RZ(0.0206)──H──────────H──╭RZ(0.97)──H──RX(1.57)──╭RZ(0.832)──RX(-1.57)──╭RZ(0.212)─────────────┤     
 1: ──

# Training

In [8]:
params = initial_params
opt = qml.GradientDescentOptimizer(stepsize=STEP_SIZE)

for i in range(STEPS):
    features = X_train[i%len(X_train)]
    expected_value = y_train[i%len(X_train)]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

KeyboardInterrupt: 

In [ ]:
print("Optimized rotation angles: {}".format(params))

# Testing

In [ ]:
prediction_class = np.array([])
prediction_values = np.array([])
for example, example_class in zip(X_test, y_test):
    predicted_value = circuit(example, params)
    prediction_values = np.append(prediction_values, predicted_value)

    if predicted_value <= 0:
        prediction_class = np.append(prediction_class, -1)
    else:
        prediction_class = np.append(prediction_class, 1)

In [ ]:
plt.xlim([-1, 20])
plt.plot(y_test, 'o')
plt.plot(prediction_values, 'x')
plt.axhline(y=0, color='r', linestyle='-')